In [6]:
library(tidyverse)
library(ggplot2)
options(repr.matrix.max.rows = 20)
library(boot)
library(broom)
library(tree)
library(digest)
library(testthat)

Registered S3 method overwritten by 'tree':
  method     from
  print.tree cli 


Attaching package: 'testthat'


The following object is masked from 'package:dplyr':

    matches


The following object is masked from 'package:purrr':

    is_null


The following objects are masked from 'package:readr':

    edition_get, local_edition


The following object is masked from 'package:tidyr':

    matches




In [17]:
# download and simplify the bee dataset

bees <- read.csv("https://files.ontario.ca/moe_mapping/downloads/4Other/PHAP/Bumble_Bee_Public_Data.csv") |>
    select(1:7, contains("total")) |>
    mutate(site_type = case_when(`Site.Type` == "Agri" ~ "Agricultural",
                                 `Site.Type` == "Agriculture" ~ "Agricultural",
                                 TRUE ~ `Site.Type`)) |>
    rename(
        county = `ï..County`,
        site_id = `MOECC.Site.Number`,
        year = Year,
        sample_date = `Sample.date`) |>
    filter(site_type != "") |>
    pivot_longer(`bimaculatus.total`:`flavidus.total`,
                 values_to = "no_of_bees", names_to = "species") |>
    select(county, year, site_id, site_type, no_of_bees, sample_date)

bees

county,year,site_id,site_type,no_of_bees,sample_date
<chr>,<int>,<int>,<chr>,<int>,<chr>
Middlesex Centre,2015,1,Agricultural,7,07/02/2015
Middlesex Centre,2015,1,Agricultural,1,07/02/2015
Middlesex Centre,2015,1,Agricultural,3,07/02/2015
Middlesex Centre,2015,1,Agricultural,0,07/02/2015
Middlesex Centre,2015,1,Agricultural,1,07/02/2015
Middlesex Centre,2015,1,Agricultural,0,07/02/2015
Middlesex Centre,2015,1,Agricultural,0,07/02/2015
Middlesex Centre,2015,1,Agricultural,1,07/02/2015
Middlesex Centre,2015,1,Agricultural,0,07/02/2015


In [19]:
# Poisson Regression (data = bees)

Poi_Reg_bee <- glm(no_of_bees ~ site_type, family = poisson, data = bees)

tidy(Poi_Reg_bee)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.15509945,0.02177517,7.122766,1.057822e-12
site_typeNatural,0.07591579,0.04099051,1.852033,6.402104e-02
site_typeSemi-natural remnant,0.20014590,0.03448863,5.803244,6.504410e-09
